In [1]:
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd
from collections import defaultdict
import numpy as np
import pickle
import os
import re
from pprint import pprint

In [2]:
top7000 = "/data/rali7/Tmp/solimanz/data/datasets/reduced7000/"
top550 = "/data/rali7/Tmp/solimanz/data/datasets/top550"
with open(os.path.join(top7000, 'train_ids.pkl'), 'rb') as f:
    ids7k = pickle.load(f)
with open(os.path.join(top7000, 'test_ids.pkl'), 'rb') as f:
    ids7k += pickle.load(f)
    
with open(os.path.join(top550, 'train_ids.pkl'), 'rb') as f:
    ids550 = pickle.load(f)
with open(os.path.join(top550, 'test_ids.pkl'), 'rb') as f:
    ids550 += pickle.load(f)

In [3]:
client = MongoClient()
candidates = client.lbj["excerpt-2017-02-20"]

In [5]:
c = candidates.find()

In [5]:
a = c.next()
a['skills'][0]['name']

'Salesforce.com'

In [16]:
b = {'_id': ObjectId('52b31a870b045119318b456e'),
 'addedValue': 46.0,
 'administrativeAreaLevel1': 'CA-ON',
 'bbox': {'east': -79.02891052158193,
  'north': 43.9788806322227,
  'south': 43.42134694377731,
  'west': -79.80369786641808},
 'city': 'TORONTO',
 'collectDate': '2015-08-29',
 'collector': ['scrapinghub'],
 'countryCode': 'CA',
 'createdDatetime': None,
 'duplicateMaster': None,
 'educations': [{'endDate': '1997',
   'name': 'MScE',
   'schoolName': 'Stanford University',
   'schoolSupplierUrl': 'http://www.linkedin.com/edu/school?id=17926',
   'sector': 'Industrial Engineering',
   'startDate': '1996'},
  {'endDate': '1993',
   'name': 'BScE',
   'schoolName': 'Princeton University',
   'schoolSupplierUrl': 'http://www.linkedin.com/edu/school?id=18867',
   'sector': 'Civil Engineering and Operations Research',
   'startDate': '1988'}],
 'experienceLevelMonthNumber': 217,
 'experiences': [{'companyName': 'TELUS',
   'company_id': '55df1c8b0b0451dc5c8bac6b',
   'function': 'Senior Vice President Consumer Marketing',
   'missions': "Support TELUS' continued growth in Consumer markets nationally across both wireless and wireline products and services",
   'place': 'toronto, ontario',
   'startDate': '2014-05'},
  {'companyName': 'TELUS',
   'company_id': '55df1c8b0b0451dc5c8bac6b',
   'function': 'VP Mobility Solutions',
   'missions': 'Support TELUS Consumer segment wireless overall business and marketing strategy, product marketing and pricing.',
   'startDate': '2008-12'},
  {'companyName': 'Barrick Gold Corporation',
   'company_id': '55df1c4d0b0451dc5c8ba610',
   'endDate': '2003',
   'function': 'Director, Continuous Improvement',
   'missions': 'Led global Operations Continuous Improvement team aimed at driving hundreds of millions of dollars of operational improvement. Pioneer in importing Lean and Six Sigma philosophies and tools into the Gold Mining sector.',
   'startDate': '2000'},
  {'companyName': 'McKinsey & Company',
   'company_id': '55df1c5f0b0451dc5c8ba7f9',
   'endDate': '2003',
   'function': 'Associate',
   'startDate': '1993'}],
 'id_scrapinghub': '45070289',
 'indexCandidate': True,
 'industriesId': [42],
 'industry': 'Telecommunications',
 'industryId': 42,
 'language': 'en',
 'lastEditedDate': '2015-08-29',
 'lat': 43.70011,
 'lng': -79.4163,
 'personalBranding_claim': 'SVP Consumer Marketing at TELUS',
 'personalBranding_pitch': 'Seasoned operations and marketing executive known for creating winning strategies and driving change in complex organizations.',
 'privacy': 1,
 'professionalData_experienceLevelId': 5,
 'relationsNumber': 500,
 'skills': [{'name': 'Continuous Improvement'},
  {'name': 'Six Sigma'},
  {'name': 'Lean Manufacturing'},
  {'name': 'Operational Excellence'},
  {'name': 'Mining'},
  {'name': 'Wireless'},
  {'name': 'Marketing Management'},
  {'name': 'Strategy'},
  {'name': 'Business Strategy'},
  {'name': 'Change Management'},
  {'name': 'Competitive Analysis'},
  {'name': 'Cross-functional Team...'},
  {'name': 'Leadership'},
  {'name': 'Management'},
  {'name': 'Product Marketing'},
  {'name': 'Strategic Planning'},
  {'name': 'Product Management'},
  {'name': 'Team Leadership'},
  {'name': 'Business Process...'},
  {'name': 'Telecommunications'},
  {'name': 'Management Consulting'},
  {'name': 'Program Management'},
  {'name': 'Project Management'},
  {'name': 'Operations Management'},
  {'name': 'Vendor Management'},
  {'name': 'Business Analysis'}],
 'supplierParty': 'linkedin',
 'updateDatetime': '2015-12-21 13:38:57'}

In [7]:
c = candidates.find({"$and": [{"language": "en"}, {"$nor": [
    {"experiences": {"$exists": False}},
    {"experiences": {"$size": 0}},
    {"experiences": {"$size": 1}}
]}]})

In [4]:
pipeline = [
            #{"$match": {"$and": [ 
            #    {"language": "en"},
            #    {"$nor": [
            #        {"experiences": {"$exists": False}},
            #        {"experiences": {"$size": 0}},
            #        {"experiences": {"$size": 1}}
            #    ]}]}},
            {"$unwind": {
                "path": "$experiences",
                "includeArrayIndex": "job_index"
                }
            },
            {"$project": {
                "industry": 1,
                "job_index": 1,
                "company_name": "$experiences.companyName",
                "place": "$experiences.place",
                "function": "$experiences.function",
                "start_date": "$experiences.startDate",
                "end_date": "$experiences.endDate"
                #"duration": "$experiences.duration"
                #"mission": "$experiences.missions"
                }}
            ]

In [6]:
agg = candidates.aggregate(pipeline)

NameError: name 'pipeline' is not defined

In [6]:
def _get_element(doc, field_name):
    if field_name in doc:
        return doc[field_name]
    return np.nan

def make_dataframe(curr):
    data = defaultdict(list)

    for doc in curr:
        data['_id'].append(doc['_id'])
        data['function'].append(_get_element(doc, 'function'))
        data['company_name'].append(_get_element(doc,'company_name'))
        data['industry'].append(_get_element(doc,'industry'))
        data['job_index'].append(doc['job_index'])        
        data['place'].append(_get_element(doc, 'place'))
        data['start_date'].append(_get_element(doc, 'start_date'))
        data['end_date'].append(_get_element(doc, 'end_date'))

    return data

In [7]:
data = make_dataframe(agg)

In [8]:
df = pd.DataFrame(data)

In [9]:
df["_id"] = df["_id"].apply(str)

In [11]:
bad_ids = df[pd.isna(df["function"])]._id.unique()

In [12]:
df.drop(df[df._id.isin(bad_ids)].index, inplace=True)

In [13]:
df.to_pickle("/data/rali7/Tmp/solimanz/data/pickles/excerpt-2017-02-20.pkl")

In [22]:
id_list = ids7k

In [23]:
ids = [ObjectId(i) for i in id_list]

In [24]:
pipeline = [
            {"$match": { "_id": { "$in": ids}}},
            {"$unwind": {
                "path": "$skills"
                }
            },
            {"$project": {
                "name": "$skills.name"
                }}
            ]
agg = candidates.aggregate(pipeline)

In [25]:
p = re.compile("(\.[a-zA-Z0-9_]*)")
skills = defaultdict(list)
for doc in agg:
    skills[str(doc['_id'])].append(doc['name'].lower().strip())

In [20]:
with open("/data/rali7/Tmp/solimanz/data/datasets/skills/skills550.pkl", "wb") as f:
    pickle.dump(skills, f)

In [26]:
with open("/data/rali7/Tmp/solimanz/data/datasets/skills/skills7000.pkl", "wb") as f:
    pickle.dump(skills, f)

In [27]:
len(skills)

664007

In [29]:
ls = [len(s) for s in skills.values()]

In [30]:
ls

[50,
 21,
 15,
 12,
 29,
 14,
 50,
 15,
 50,
 48,
 15,
 26,
 27,
 17,
 35,
 35,
 15,
 15,
 49,
 19,
 15,
 29,
 24,
 12,
 25,
 50,
 15,
 26,
 15,
 15,
 16,
 19,
 20,
 28,
 35,
 20,
 33,
 15,
 15,
 40,
 15,
 50,
 14,
 15,
 11,
 15,
 15,
 15,
 27,
 50,
 50,
 50,
 47,
 15,
 15,
 15,
 22,
 23,
 19,
 27,
 15,
 15,
 50,
 6,
 20,
 16,
 25,
 22,
 25,
 50,
 15,
 15,
 50,
 27,
 15,
 15,
 15,
 50,
 15,
 48,
 15,
 15,
 16,
 20,
 15,
 44,
 15,
 40,
 33,
 29,
 40,
 6,
 18,
 15,
 50,
 13,
 50,
 8,
 15,
 15,
 14,
 12,
 17,
 1,
 15,
 14,
 9,
 18,
 15,
 15,
 15,
 16,
 19,
 15,
 14,
 18,
 9,
 12,
 17,
 15,
 8,
 10,
 15,
 15,
 8,
 18,
 11,
 12,
 15,
 15,
 9,
 17,
 15,
 21,
 18,
 15,
 15,
 11,
 15,
 9,
 12,
 5,
 15,
 15,
 16,
 15,
 12,
 15,
 15,
 12,
 15,
 15,
 16,
 15,
 16,
 15,
 15,
 15,
 18,
 15,
 15,
 15,
 15,
 11,
 15,
 15,
 16,
 12,
 15,
 6,
 18,
 15,
 15,
 15,
 15,
 15,
 10,
 15,
 15,
 15,
 15,
 15,
 21,
 8,
 17,
 15,
 6,
 8,
 16,
 5,
 7,
 13,
 15,
 8,
 15,
 15,
 15,
 17,
 15,
 3,
 15,
 15,
 15,
 13,

In [44]:
len(id_list)

837999

In [56]:
len(all_ids)

664007

In [57]:
c = candidates.find({ "_id": { "$in": ids}})